# Installation of required libraries

In [1]:
! pip install transformers                               
# sentencepiece library for t5 transformers
! pip install sentencepiece
! pip install sumy

     |████████████████████████████████| 3.1 MB 7.6 MB/s 
     |████████████████████████████████| 56 kB 2.7 MB/s 
     |████████████████████████████████| 596 kB 41.9 MB/s 
     |████████████████████████████████| 3.3 MB 26.0 MB/s 
     |████████████████████████████████| 895 kB 42.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 8.5 MB/s 
     |████████████████████████████████| 87 kB 4.6 MB/s 
     |████████████████████████████████| 10.1 MB 36.0 MB/s 
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21711 sha256=8003ab6555963c089c65e6dcbdab22abeb3a9cc6b1653f962210624509a1fe21
  Stored in directory: /root/.cache/pip/wheels/d4/bf/51/81d27ad638e1a6dca4f362ecc33d1e2c764b8ea7ec751b8fc1
  Created wheel for pycountry: filename=pycountry-20.7.3-py2.py3-none-any.whl size=10746883 sha256=d83c9fcedfabdac480f

In [2]:
! nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



# Preprocessing for the text summarizer for sceintific research articles in .tex file

In [3]:
import re
def clean_latext_content(text):
  clean_text = re.sub(r'\{[^)]*\}', '', text)
  clean_text2 = clean_text.replace("\cite", "").replace("\n","").replace("\ref", "").replace("\textit","").replace('\\',"")
  return clean_text2

# Data: Scentific research papers in Computer Science  

In [4]:
abstract_content="""

Per-instance Algorithm Selection (AS) denotes the problem of recommending an algorithm that appears to be most suitable for a given instance of a problem class. The suitability is assessed with respect to a specific performance criterion, such as solution quality or runtime.
The latter is of special interest when dealing with computationally hard problems, such as the Boolean satisfiability problem (SAT) \cite{xu2007satzilla,satzilla11_xu2011hydra}, the traveling salesperson problem (TSP) \cite{DBLP:conf/ictai/PiheraM14}, or constraint satisfaction problems (CSP) \cite{lobjois1998branch}, just to name a few.
For this kind of problems, different solvers or optimizers have been developed, which build on different heuristics to exploit certain structures inherent to the problem instances. As the heuristics are complementary in some sense, choosing the solver or optimizer on a per-instance basis can drastically improve the overall performance \cite{survey_kerschkeHNT19}.

A common approach to AS is the use of machine learning methods to predict the runtime of algorithms on unseen problem instances.  One major challenge for such approaches concerns the training data, parts of which are usually censored.
The censoring has its root in the way the training data is generated. To assess the true runtime of an algorithm for a problem instance, it is simply run on that particular instance. However, for specifically hard instances, the algorithms may take days, weeks, or even years before returning a solution.
To keep the computational effort reasonable, a cutoff time is set at which the execution of still running algorithms is aborted.
In the commonly used AS benchmarking suite ASlib \cite{bischl_aslib_2016}, up to 70\% of the instances in the training data are censored (cf.\ Table \ref{tab:aslib_overview}).

To deal with censored data, several approaches consider them as missing values and apply imputation techniques to replace these values \cite{regression_with_censored_data_schmee1979simple,xu2007satzilla}. Alternatively, techniques from the field of survival analysis have been proposed \cite{gagliolo2006dynamic,gagliolo2010algorithm,tornede2020run2survive}. While the latter approach appropriately captures the information provided by a censored observation, namely that the runtime exceeds the cutoff time $C$, the former turns it into unduly precise information and comes with the risk of incorporating a bias in the learning process.
%based on the observations on other instances, thereby distorting the actual ground truth for training. More specifically, the information that is available when an instance has a censored label is that the runtime exceeds the cutoff time $C$, i.e., the ground truth value lies in the interval $(C,\infty)$.

In this paper, we propose to consider AS as a so-called superset learning problem \cite{superset_learning_hullermeierC15}, where the learner induces a (precise) predictive model from possibly imprecise training data. More concretely, we learn a regression model for runtime prediction from training instances that are either precisely labeled or labeled with an interval of the form $(C, \infty)$.
This is not only a just representation of the (weak albeit non-void) information that the true runtime exceeds $C$, but, as will be seen, also offers an efficient way for handling censored data. In our experimental evaluation, we show that methods based on superset learning can induce (linear) models that outperform na\"ive strategies for dealing with censored data in AS.
"""

In [5]:
cleaned_latext_text = clean_latext_content(abstract_content)

In [6]:
cleaned_latext_text

'Per-instance Algorithm Selection (AS) denotes the problem of recommending an algorithm that appears to be most suitable for a given instance of a problem class. The suitability is assessed with respect to a specific performance criterion, such as solution quality or runtime.The latter is of special interest when dealing with computationally hard problems, such as the Boolean satisfiability problem (SAT) , the traveling salesperson problem (TSP) , or constraint satisfaction problems (CSP) ).To deal with censored data, several approaches consider them as missing values and apply imputation techniques to replace these values . While the latter approach appropriately captures the information provided by a censored observation, namely that the runtime exceeds the cutoff time $C$, the former turns it into unduly precise information and comes with the risk of incorporating a bias in the learning process.%based on the observations on other instances, thereby distorting the actual ground truth

# Class Text Summarizer:


>  Class return the text summarisation performed on given text  using diffferent different state-of-the-art text summmarizer



In [7]:
import torch
import sumy
import gensim
from gensim.summarization import summarize
import nltk
import re
nltk.download('punkt')
from transformers import BartForConditionalGeneration, BartTokenizer, BartConfig,BartModel
from transformers import GPT2Tokenizer,GPT2LMHeadModel
from transformers import XLMWithLMHeadModel, XLMTokenizer
from transformers import T5Tokenizer, T5Config, T5ForConditionalGeneration
from transformers import BigBirdPegasusForConditionalGeneration, AutoTokenizer
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

from bs4 import BeautifulSoup 
# Import the LexRank summarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
# Importing the parser and tokenizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
# Import the LexRank summarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
lex_rank_summarizer = LexRankSummarizer() 
from sumy.summarizers.lsa import LsaSummarizer
lsa_summarizer=LsaSummarizer()
# Parsing the text string using PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser


class TextSummarizer:
  def __init__(self):
    # Instantiating the model and tokenizer with gpt-2 and bart
    self.tokenizer_gpt2=GPT2Tokenizer.from_pretrained('gpt2')
    self.model_gpt2=GPT2LMHeadModel.from_pretrained('gpt2')
    # Instantiating baer model 
    self.tokenizer_bart=BartTokenizer.from_pretrained('facebook/bart-large-cnn')
    self.model_bart=BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
    # Instantiating the model and tokenizer t5
    self.model_t5 = T5ForConditionalGeneration.from_pretrained('t5-small')
    self.tokenizer_t5 = T5Tokenizer.from_pretrained('t5-small')
    # Instantiating the model and tokenizer google_bigbird
    self.model_BigBird = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-arxiv", attention_type="original_full")
    self.tokenizer_BigBird = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-arxiv")
    # by default encoder-attention is `block_sparse` with num_random_blocks=3, block_size=64
    
    self.tokenizer_Pegasus = PegasusTokenizer.from_pretrained('google/pegasus-xsum')
    self.model_Pegasus = PegasusForConditionalGeneration.from_pretrained('google/pegasus-xsum')


  def text_summarizer(self, text):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    text_summary_list = []
    text_summary_dict = {}

    text_summary_dict['_text_'] = text
    # Encoding the inputs and passing them to model.generate()
    inputs = self.tokenizer_bart.batch_encode_plus([text],return_tensors='pt')
    summary_ids = self.model_bart.generate(inputs['input_ids'], early_stopping=True,min_length = 50, max_length=75)
    bart_summary = self.tokenizer_bart.decode(summary_ids[0], skip_special_tokens=True)
    print(bart_summary)
    text_summary_list.append(bart_summary)
    text_summary_dict['bart_summary'] = bart_summary

    inputs = self.tokenizer_BigBird(text, return_tensors='pt')
    summary_ids = self.model_BigBird.generate(**inputs,min_length=50, max_length=75)
    summary_BigBird = self.tokenizer_BigBird.batch_decode(summary_ids, skip_special_tokens=True)
    text_summary_list.append(summary_BigBird)
    text_summary_dict['summary_BigBird'] = summary_BigBird
    print(summary_BigBird)
    
    # GPL Summary
    inputs=self.tokenizer_gpt2.batch_encode_plus([text],return_tensors='pt',max_length=512)
    summary_ids=self.model_gpt2.generate(inputs['input_ids'],early_stopping=True)
    GPT_summary=self.tokenizer_gpt2.decode(summary_ids[0],skip_special_tokens=True)
    # print(GPT_summary)
    text_summary_list.append(GPT_summary)
    text_summary_dict['gpt_summary'] = GPT_summary
    # Concatenating the word "summarize:" to raw text
    # google pegasus summarization
    inputs = self.tokenizer_Pegasus([text], truncation=True, padding='longest', return_tensors="pt")
    summary_ids = self.model_Pegasus.generate(inputs['input_ids'], min_length=50, max_length=75)
    summary_google_pegasus = self.tokenizer_Pegasus.batch_decode(summary_ids, skip_special_tokens=True)
    text_summary_list.append(summary_google_pegasus)
    text_summary_dict['summary_google_pegasus'] = summary_google_pegasus
    
    t5_text = "summarize:" + text
    # encoding the input text
    input_ids=self.tokenizer_t5.encode(t5_text, return_tensors='pt')
    summary_ids = self.model_t5.generate(input_ids,early_stopping=True, min_length = 50, max_length=75)
    t5_summary = self.tokenizer_t5.decode(summary_ids[0], skip_special_tokens=True)
    text_summary_list.append(t5_summary)
    text_summary_dict['t5_summary'] = t5_summary
    # gensim_summary
    summary_genensim = summarize(text)
    text_summary_dict['gensim_summary'] = summary_genensim
    text_summary_list.append(summary_genensim)
    my_parser = PlaintextParser.from_string(text,Tokenizer('english'))
    lexrank_summary_sentences = lex_rank_summarizer(my_parser.document,sentences_count=3)
    lexrank_summary = ""
    for sentence in lexrank_summary_sentences:
      lexrank_summary= lexrank_summary + " "+ str(sentence)
    text_summary_list.append(lexrank_summary)
    text_summary_dict['lexrank_summary'] = lexrank_summary
    # creating the lsa summarizer
    parser=PlaintextParser.from_string(text,Tokenizer('english'))
    lsa_summary_sentences= lsa_summarizer(parser.document,3)
    lsa_summary =""
    #  lsa summary
    for sentence in lsa_summary_sentences:
        lsa_summary= lsa_summary+" "+ str(sentence)
    text_summary_list.append(lsa_summary)
    text_summary_dict['lsa_summary'] = lsa_summary
    return text_summary_list, text_summary_dict

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
textSummarizer = TextSummarizer()

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

In [9]:
text_summary_list, text_summary_dict = textSummarizer.text_summarizer(cleaned_latext_text)

Per-instance Algorithm Selection (AS) denotes the problem of recommending an algorithm that appears to be most suitable for a given instance of a problem class. To deal with censored data, several approaches consider them as missing values and apply imputation techniques to replace these values. In this paper, we propose to consider AS as a so-called superset learning problem


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 382, but ``max_length`` is set to 20. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.


['we consider as socalled superset learning problem : where the learning process is to learn a regression model from possibly imprecise training data.<n> specifically, we consider as socalled superset learning problem : where the learning process is to learn a predictive model from possibly imprecise training data.<n> this paper we propose to consider as socalled superset learning problem']


In [10]:
text_summary_dict

{'_text_': 'Per-instance Algorithm Selection (AS) denotes the problem of recommending an algorithm that appears to be most suitable for a given instance of a problem class. The suitability is assessed with respect to a specific performance criterion, such as solution quality or runtime.The latter is of special interest when dealing with computationally hard problems, such as the Boolean satisfiability problem (SAT) , the traveling salesperson problem (TSP) , or constraint satisfaction problems (CSP) ).To deal with censored data, several approaches consider them as missing values and apply imputation techniques to replace these values . While the latter approach appropriately captures the information provided by a censored observation, namely that the runtime exceeds the cutoff time $C$, the former turns it into unduly precise information and comes with the risk of incorporating a bias in the learning process.%based on the observations on other instances, thereby distorting the actual g

# Dumping the result into json files

In [13]:
file_path = r"/content/drive/MyDrive/TextSummarizationResults/summary_results.json"

In [14]:
import json
with open(file_path, 'w') as fp:
    json.dump(text_summary_dict, fp, indent=4)